In [1]:
from io import BytesIO
import matplotlib.dates as dates
import requests
import pandas as pd
import re
import scipy.optimize
import numpy as np
from matplotlib import pyplot as plt
from openpyxl import load_workbook
import ipywidgets as widgets

%matplotlib ipympl

In [2]:
response = requests.get("https://www.arcgis.com/sharing/rest/content/items/e5fd11150d274bebaaf8fe2a7a2bda11/data")
io = BytesIO(response.content)

In [3]:
workbook = load_workbook(io, read_only=True)

In [4]:
header, *rows = workbook.worksheets[0].values
df = pd.DataFrame({c: b for c, b in zip(header, zip(*rows))})
df.tail()

,DateVal,CMODateCount,CumCases
36,2020-03-07,46,=C37+B38
37,2020-03-08,65,=C38+B39
38,2020-03-09,50,=C39+B40
39,2020-03-10,52,=C40+B41
40,2020-03-11,83,=C41+B42


In [5]:
df = df.rename(columns={'CMODateCount': 'new cases', 'CumCases': 'cases', 'DateVal': 'date'})
df['cases'] = df['new cases'].cumsum()
df.tail()

,date,new cases,cases
36,2020-03-07,46,206
37,2020-03-08,65,271
38,2020-03-09,50,321
39,2020-03-10,52,373
40,2020-03-11,83,456


In [6]:
df.plot('date', 'cases', kind='scatter', title='UK Cases');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def fit_func(x, a, m, c):
    return a * np.exp(m*x + c)

In [8]:
popt = .3, .13, -2

In [9]:
days = df['date'].dt.dayofyear
popt, pcov = scipy.optimize.curve_fit(fit_func, days, df['cases'], p0=(0.3, 0.13, -2))

In [10]:
plt.plot(df['date'], fit_func(days, *popt), 'g');

In [12]:
def predict(change):
    n_days = change['new']
    t_projection = days.to_numpy()[-1] + np.arange(n_days)
    dt_projection = pd.to_datetime(t_projection - 1, unit='D', origin=pd.Timestamp("2020-01-01"))
    
    line_predict.set_data(dt_projection, fit_func(t_projection, *popt))
    
    ax = plt.gca()
    ax.relim()
    ax.autoscale()

In [13]:
line_predict, = plt.plot(df['date'], df['cases'], 'r--');

In [14]:
n_days = widgets.IntSlider(min=0, max=40, description="Days")
n_days.observe(predict, 'value')
n_days

IntSlider(value=0, description='Days', max=40)